<a href="https://colab.research.google.com/github/ahmedm-g/NewsClassifier/blob/main/news_classifier_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [118]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [119]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [120]:
def get_data():
  url = 'https://raw.githubusercontent.com/ahmedm-g/NewsClassifier/main/data_train_news_bbc.csv'
  return pd.read_csv(url)
data = get_data()

In [121]:
print(data.head())

   ArticleId                                               Text  Category
0       1833  worldcom ex-boss launches defence lawyers defe...  business
1        154  german business confidence slides german busin...  business
2       1101  bbc poll indicates economic gloom citizens in ...  business
3       1976  lifestyle  governs mobile choice  faster  bett...      tech
4        917  enron bosses in $168m payout eighteen former e...  business


In [122]:
data.shape

(1490, 3)

In [123]:
import re
import string
import sys
import argparse
from nltk.corpus import stopwords

english_stops = stopwords.words('english')
english_stops += list(string.punctuation)

In [124]:
def lower(text):
  return text.lower()
data['Text'] = data['Text'].apply(lower)
data['Text'].head()

0    worldcom ex-boss launches defence lawyers defe...
1    german business confidence slides german busin...
2    bbc poll indicates economic gloom citizens in ...
3    lifestyle  governs mobile choice  faster  bett...
4    enron bosses in $168m payout eighteen former e...
Name: Text, dtype: object

In [125]:
from nltk.tokenize import word_tokenize

def remove_stopwords(text):
  tokens = word_tokenize(text)
  return ' '.join([token for token in tokens if token not in english_stops])

data['Text'] = data['Text'].apply(remove_stopwords)
data['Text'].head()

0    worldcom ex-boss launches defence lawyers defe...
1    german business confidence slides german busin...
2    bbc poll indicates economic gloom citizens maj...
3    lifestyle governs mobile choice faster better ...
4    enron bosses 168m payout eighteen former enron...
Name: Text, dtype: object

In [126]:
from nltk.stem import WordNetLemmatizer

def lemmatize_text(text):
  lemmatizer = WordNetLemmatizer()
  lemmatized_words = [lemmatizer.lemmatize(word) for word in text]
  return ' '.join(lemmatized_words)

# data['Text'] = data['Text'].apply(lemmatize_text)
data['Text'].head()

0    worldcom ex-boss launches defence lawyers defe...
1    german business confidence slides german busin...
2    bbc poll indicates economic gloom citizens maj...
3    lifestyle governs mobile choice faster better ...
4    enron bosses 168m payout eighteen former enron...
Name: Text, dtype: object